In [1]:
import ray
#ray.shutdown()
#ray.init()

In [2]:
from files import BCTrade_utils
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()
name ='Bitstamp_2'
dataset = Dataset.get_by_name(ws, name=name)
env = BCTrade_utils.BCTrade(dataset.to_pandas_dataframe(),singlerun=False,spreadfactor=0.0)
experiment_name = BCTrade_utils.registerBCTrade(name)
from ray.tune.registry import register_env
def env_creator(env_config):
    return BCTrade_utils.BCTrade(dataset.to_pandas_dataframe())
register_env("BCTrade-Bitstamp_2", env_creator)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [3]:
#datastore = ws.get_default_datastore()
#checkpoint_ds = Dataset.File.from_files(path=[(datastore,'checkpoints_BCTrade-Btc_with_indicators_1641664748_e3baadf3_head/checkpoint_370')])
#checkpoint = checkpoint_ds.as_named_input('test')
#checkpoint.as_mount('xxx')
#checkpoint = '/tmp/checkpoint_370/checkpoint-370'
#Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_100
checkpoint = '/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/'
checkpoint+='logs/PPO/PPO_BCTrade-Bitstamp_1_3e036a2e_2022-03-08_18-54-54fnue5ctr/checkpoint_1032/checkpoint-1032'

#checkpoint = 'logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_110/checkpoint-110'

In [4]:
import ray
import ray.rllib.agents.ppo as ppo

#ray.init()
#config = {
  #"callbacks": {
  #  "on_train_result": "<function on_train_result at 0x7fca737a8e18>"
  #},
#  "env": "BCTrade-Btc_with_indicators",
#  "num_gpus": 0,
#  "num_workers": 1
#}

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
#config["eager"] = False
config["env"] = "BCTrade-Bitstamp_2"
new_trainer = ppo.PPOTrainer(config=config,env="BCTrade-Bitstamp_2")
new_trainer


2022-03-11 21:29:08,194	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=16859) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
(RolloutWorker pid=16859)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=16859) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(RolloutWorker pid=16859) Instructions for updating:
(RolloutWorker pid=16859) If using Keras pass *_constraint arguments to layers.
(RolloutWorker pid=16859) 2022-03-11 21:29:37,705	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


PPO

In [5]:
import pickle
with open(checkpoint, "rb") as f:
        extra_data = pickle.load(f)

extra_data['worker']
weights=pickle.loads(extra_data['worker'])
weights['state']['default_policy']
new_trainer.get_policy().set_weights(weights['state']['default_policy'])
#new_trainer.get_policy().set_state(weights['state'])
#new_trainer.restore(checkpoint)
#ray.tune.run(ppo.PPOTrainer, config=config, restore='/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_95fc9bdc_2022-01-10_21-38-0176w5xe80/checkpoint_822')

In [6]:
#obs = env.initialize()
obs = env.reset()
rewardsum=0
c=0
while True:
	action = new_trainer.compute_action(obs, explore=False)
	if obs[0]==1 and action ==2:
		env.render()
	obs, reward, done, _ = env.step(action)
	rewardsum+=reward
	c+=1
	if done:
		break
env.render()
print(rewardsum,rewardsum/c,c)


1000.0
999.9966307121537
1000.6555709819803
1000.835734581302
1000.556054107083
999.0736084586284
998.8371096590865
998.5472831810804
998.6298151971687
998.8512111988149
999.7316779336408
1000.9923104126999
1002.1068603022068
1001.4608488744591
1001.1379631748848
1001.9506471942192
1003.0753412891861
1002.4051395624342
1002.3903679478866
1004.1990896682893
1004.2809976545993
1004.5436506034679
1005.1959289063379
1005.2031106515724
1005.1870656800372
1007.2505091972772
1006.3067353872884
1006.6220121075422
1006.993452024088
1006.7446856894449
1006.6188508906729
1006.4067606242908
1006.376308947812
1007.1217102073713
1007.6425538116339
1007.2253012404243
1006.9945740651743
1006.5672697850897
1007.3269732218426
1007.3206242930165
1009.026565527438
1007.0837188192115
1012.9062991046643
1015.9858537460555
1016.1750332669648
1015.9743557926816
1017.1628869313154
1015.9195474816647
1016.3702302309362
1016.75559719931
1016.9254169973171
1017.7161954749267
1018.0691158524834
1017.9158698670387


2022-03-11 21:29:41,251	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


KeyboardInterrupt: 